In [6]:
# 無量空処

import cv2
import mediapipe as mp

# MediaPipeのセットアップ
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7)

# 画像の読み込み
overlay_image = cv2.imread("image/muryokusho.jpg")  # 合成する画像

def resize_image_to_frame(image, width, height):
    """画像をウィンドウ全体のサイズにリサイズする関数"""
    return cv2.resize(image, (width, height))

def blend_images(background, overlay, alpha):
    """
    背景画像に透過エフェクトで画像を重ねる
    - background: 背景画像
    - overlay: 合成する画像
    - alpha: 透明度（0.0～1.0）
    """
    blended = cv2.addWeighted(overlay, alpha, background, 1 - alpha, 0)
    return blended

def is_thumb_near_ring_joint(landmarks):
    """
    親指の先が薬指の第三関節に近いかを判定
    """
    thumb_tip = landmarks[4]
    ring_joint = landmarks[14]

    # 距離を計算
    distance = ((thumb_tip.x - ring_joint.x)**2 + (thumb_tip.y - ring_joint.y)**2)**0.5
    return distance < 0.05  # 閾値を設定

cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # フレームをRGB形式に変換してMediaPipeで処理
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb_frame)

    # フレームサイズを取得
    frame_height, frame_width = frame.shape[:2]

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # ランドマークを映像上に描画
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            landmarks = hand_landmarks.landmark
            if is_thumb_near_ring_joint(landmarks):
                # 条件を満たした場合、画像をリサイズしてフレームに合成
                resized_image = resize_image_to_frame(overlay_image, frame_width, frame_height)
                frame = blend_images(frame, resized_image, 0.7)  # 透明度70%で合成

    # ウィンドウにフレームを表示
    cv2.imshow('Domain Expansion Application', frame)

    # 'q'キーで終了
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [8]:
# 伏魔御厨子

import cv2
import mediapipe as mp
import random

# MediaPipeのセットアップ
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7)

# パーティクル設定
particles = []

def load_image(path):
    """画像を読み込む関数"""
    image = cv2.imread(path)
    if image is None:
        raise FileNotFoundError(f"Image not found: {path}")
    return image

def resize_image(image, width, height):
    """画像をウィンドウ全体のサイズにリサイズ"""
    return cv2.resize(image, (width, height))

def blend_images(background, overlay, alpha):
    """背景画像に透過エフェクトで画像を重ねる"""
    return cv2.addWeighted(overlay, alpha, background, 1 - alpha, 0)

def calculate_distance(point1, point2):
    """2点間の距離を計算"""
    return ((point1.x - point2.x) ** 2 + (point1.y - point2.y) ** 2) ** 0.5

def is_special_gesture(landmarks_list):
    """
    両手の条件を確認
    """
    if len(landmarks_list) != 2:
        return False  # 両手が検出されていない場合は条件を満たさない

    # 左手と右手のランドマークを取得
    left_hand = landmarks_list[0]
    right_hand = landmarks_list[1]

    def check_single_hand_conditions(landmarks):
        """
        片手の条件1と条件3を確認
        """
        middle_tip = landmarks[12]  # 中指先端
        ring_tip = landmarks[16]  # 薬指先端
        middle_base = landmarks[9]  # 中指基部
        ring_base = landmarks[13]  # 薬指基部
        thumb_knuckle = landmarks[3]  # 親指第2関節
        thumb_tip = landmarks[4]  # 親指先端
        index_knuckle = landmarks[6]  # 人差し指第2関節
        index_tip = landmarks[8]  # 人差し指先端
        pinky_knuckle = landmarks[18]  # 小指第2関節
        pinky_tip = landmarks[20]  # 小指先端

        # 条件1: 中指と薬指が伸びている（距離の閾値を緩和）
        middle_extended = calculate_distance(middle_tip, middle_base) > 0.03  # 元は0.04
        ring_extended = calculate_distance(ring_tip, ring_base) > 0.03  # 元は0.04

        # 条件3: 親指、人差し指、小指が曲がっている（距離の閾値を緩和）
        thumb_bent = calculate_distance(thumb_tip, thumb_knuckle) < 0.06  # 元は0.05
        index_bent = calculate_distance(index_tip, index_knuckle) < 0.06  # 元は0.05
        pinky_bent = calculate_distance(pinky_tip, pinky_knuckle) < 0.06  # 元は0.05

        return middle_extended and ring_extended and thumb_bent and index_bent and pinky_bent

    # 左手と右手の条件1と条件3を確認
    left_hand_condition = check_single_hand_conditions(left_hand)
    right_hand_condition = check_single_hand_conditions(right_hand)

    # 条件2: 左手と右手の中指と薬指の先端が近い（距離の閾値を緩和）
    left_middle_tip = left_hand[12]
    right_middle_tip = right_hand[12]
    left_ring_tip = left_hand[16]
    right_ring_tip = right_hand[16]

    middle_fingers_close = calculate_distance(left_middle_tip, right_middle_tip) < 0.06  # 元は0.05
    ring_fingers_close = calculate_distance(left_ring_tip, right_ring_tip) < 0.06  # 元は0.05

    # 両手で条件1、条件3を満たし、かつ条件2を満たす場合
    return left_hand_condition and right_hand_condition and middle_fingers_close and ring_fingers_close

def add_particles(frame_width, frame_height, count=10):
    """パーティクルをランダムに生成"""
    colors = [(100, 0, 0), (0, 0, 100), (50, 0, 50)]  # 暗い赤、青、紫
    for _ in range(count):
        particles.append({
            "x": random.randint(0, frame_width),
            "y": random.randint(0, frame_height),
            "size": random.randint(3, 8),
            "dx": random.uniform(-2, 2),
            "dy": random.uniform(-2, 2),
            "color": random.choice(colors),
            "alpha": random.uniform(0.5, 0.8)  # パーティクルの透明度
        })

def update_and_draw_particles(frame):
    """パーティクルを描画し、更新"""
    global particles
    updated_particles = []
    for particle in particles:
        overlay = frame.copy()
        cv2.circle(overlay, (int(particle["x"]), int(particle["y"])), int(particle["size"]), particle["color"], -1)
        # パーティクルを薄く描画（透明度適用）
        cv2.addWeighted(overlay, particle["alpha"], frame, 1 - particle["alpha"], 0, frame)
        particle["x"] += particle["dx"]
        particle["y"] += particle["dy"]
        particle["size"] -= 0.1
        if particle["size"] > 0:
            updated_particles.append(particle)
    particles = updated_particles

def process_frame(frame, overlay_image, landmarks_list):
    """フレームを処理し、必要に応じて画像合成とパーティクル描画"""
    frame_height, frame_width = frame.shape[:2]
    if is_special_gesture(landmarks_list):
        resized_image = resize_image(overlay_image, frame_width, frame_height)
        frame = blend_images(frame, resized_image, 0.7)
        add_particles(frame_width, frame_height)
    update_and_draw_particles(frame)
    return frame

def main():
    """メイン処理"""
    try:
        overlay_image = load_image("image/hukumamizushi.jpg")
    except FileNotFoundError as e:
        print(e)
        return

    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Error: Unable to access the camera.")
        return

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(rgb_frame)

        if results.multi_hand_landmarks:
            landmarks_list = [hand_landmarks.landmark for hand_landmarks in results.multi_hand_landmarks]
            frame = process_frame(frame, overlay_image, landmarks_list)

            # ランドマークを映像上に描画
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

        cv2.imshow("Domain Expansion Application", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()
